### VGG Net

[1. VGG Net 논문](https://arxiv.org/pdf/1409.1556.pdf)
2. 2014 ILSVRC 2nd Place
3. VGG -1 16
4. Convolution Layer
5. Maxpooling Layer
6. Fully Connected Layer
![image](https://qph.fs.quoracdn.net/main-qimg-83c7dee9e8b039c3ca27c8dd91cacbb4)

### 1. Setting

#### 1-1. Import Required Libraries

In [43]:
import numpy as np
import torch  
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

#### 1-2 Hyperparameters

In [44]:
batch_size = 1
learning_rate = 0.001
num_epoch = 100

### 2. Data

#### 2-1 Download

In [45]:
img_dir = './CIFAR_Data'
train = dset.CIFAR10(img_dir, train=True, target_transform = None, 
                        transform = transforms.Compose([
                         transforms.Scale(256),
                         transforms.RandomSizedCrop(224),
                         transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                     ]))

In [46]:
len(train)

50000

In [47]:
train.__getitem__(0)[0].size()

torch.Size([3, 224, 224])

#### 2-2. Data Batch

In [48]:
train_loader = data.DataLoader(img_dir, batch_size=batch_size, shuffle = True, num_workers=2)

### 3. Model

#### 3-1 Basic Block

In [53]:
#Conv_2_block
def Conv_2_block(in_channel, out_channel):
    model=nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
    )
    
    return model

#Conv_3_block
def Conv_3_block(in_channel, out_channel):
    model = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
    )
    
    return model


#### 3-2 VGG Net

In [57]:
class VGG(nn.Module):
    
    def __init__(self, base_dim, num_classes=2):
        super(VGG, self).__init__()
        self.Conv = nn.Sequential(
                                Conv_2_block(3, base_dim),
                                Conv_2_block(base_dim, 2*base_dim),
                                Conv_3_block(2*base_dim, 4*base_dim),
                                Conv_3_block(4*base_dim, 8*base_dim),
                                Conv_3_block(8*base_dim, 8*base_dim),
        )
        self.fc_layer = nn.Sequential(
                                nn.Linear(8*base_dim*7*7, 100),
                                nn.ReLU(),
                                nn.Dropout(),
                                nn.Linear(100, 20),
                                nn.ReLU(),
                                nn.Dropout(),
                                nn.Linear(20, 10),
        )
        
    def forward(self, x):
        x = self.Conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x
    
model = VGG(base_dim=64)

for i in model.named_children():
    print(i) 

('Conv', Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequen

### 4. Optimizer & Loss

In [58]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### 5. Train

In [59]:
for epoch in range(num_epoch):
    for i, data in enumerate(train_loader):
        images, labels = data
        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        y_pred = model(images)
        loss=loss_func(y_pred, labels)
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            print(epoch, i, loss)

ValueError: not enough values to unpack (expected 2, got 1)